In [2]:
import pandas as pd

In [32]:
train_data = pd.read_csv('train.csv', header=0)
train_data = train_data.drop(columns=['Id', 'Name', 'Outcome Time'])
train_data = train_data.drop(columns=['Found Location'])
train_data = train_data.dropna()
print(train_data.columns)
train_data = pd.get_dummies(train_data, columns=['Intake Condition', 'Intake Type', 'Animal Type', 'Sex upon Intake'])
print(train_data.columns)
train_data.head()

Index(['Intake Time', 'Intake Type', 'Intake Condition', 'Animal Type',
       'Sex upon Intake', 'Age upon Intake', 'Breed', 'Color', 'Date of Birth',
       'Outcome Type'],
      dtype='object')
Index(['Intake Time', 'Age upon Intake', 'Breed', 'Color', 'Date of Birth',
       'Outcome Type', 'Intake Condition_Aged', 'Intake Condition_Agonal',
       'Intake Condition_Behavior', 'Intake Condition_Congenital',
       'Intake Condition_Feral', 'Intake Condition_Injured',
       'Intake Condition_Med Attn', 'Intake Condition_Med Urgent',
       'Intake Condition_Medical', 'Intake Condition_Neonatal',
       'Intake Condition_Neurologic', 'Intake Condition_Normal',
       'Intake Condition_Nursing', 'Intake Condition_Other',
       'Intake Condition_Parvo', 'Intake Condition_Pregnant',
       'Intake Condition_Sick', 'Intake Condition_Space',
       'Intake Condition_Unknown', 'Intake Type_Abandoned',
       'Intake Type_Euthanasia Request', 'Intake Type_Owner Surrender',
       'Intake

,Intake Time,Age upon Intake,Breed,Color,Date of Birth,Outcome Type,Intake Condition_Aged,Intake Condition_Agonal,Intake Condition_Behavior,Intake Condition_Congenital,...,Intake Type_Public Assist,Intake Type_Stray,Intake Type_Wildlife,Animal Type_Cat,Animal Type_Dog,Sex upon Intake_Intact Female,Sex upon Intake_Intact Male,Sex upon Intake_Neutered Male,Sex upon Intake_Spayed Female,Sex upon Intake_Unknown
0,07/05/2015 12:59:00 PM,8 years,English Springer Spaniel,White/Liver,07/05/2007,Return to Owner,False,False,False,False,...,False,True,False,False,True,False,False,False,True,False
1,04/14/2016 06:43:00 PM,11 months,Basenji Mix,Sable/White,04/17/2015,Return to Owner,False,False,False,False,...,False,True,False,False,True,False,True,False,False,False
2,05/12/2022 12:23:00 AM,2 years,Domestic Shorthair,Orange Tabby,05/12/2020,Transfer,False,False,False,False,...,True,False,False,True,False,False,False,True,False,False
3,02/18/2017 12:46:00 PM,2 years,Labrador Retriever Mix,Chocolate,02/18/2015,Return to Owner,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
4,04/16/2019 09:53:00 AM,6 years,Great Dane Mix,Black,06/03/2012,Return to Owner,False,False,False,False,...,True,False,False,False,True,False,False,True,False,False


In [15]:
for col in train_data.columns:
    uniques = train_data[col].dropna().unique()
    print(f"Column: {col}")
    print(uniques[:20])  # limit to first 20 unique values
    print(f"Total unique (non-null): {len(uniques)}")
    print("-" * 40)

Column: Id
['A706918' 'A724273' 'A857105' 'A743852' 'A635072' 'A844350' 'A708452'
 'A818975' 'A774147' 'A731435' 'A760053' 'A696408' 'A790209' 'A856107'
 'A657188' 'A783861' 'A707658' 'A895185' 'A736287' 'A881801']
Total unique (non-null): 111157
----------------------------------------
Column: Intake Time
['07/05/2015 12:59:00 PM' '04/14/2016 06:43:00 PM'
 '05/12/2022 12:23:00 AM' '02/18/2017 12:46:00 PM'
 '04/16/2019 09:53:00 AM' '10/15/2021 11:40:00 AM'
 '07/30/2015 02:37:00 PM' '06/18/2020 02:53:00 PM'
 '06/11/2018 07:45:00 AM' '08/08/2016 05:52:00 PM'
 '10/11/2017 03:46:00 PM' '02/04/2015 12:58:00 PM'
 '03/06/2019 02:31:00 PM' '04/28/2022 11:56:00 AM'
 '11/10/2013 05:19:00 PM' '11/07/2018 03:53:00 PM'
 '07/15/2015 05:43:00 PM' '12/23/2023 12:10:00 PM'
 '10/08/2016 11:53:00 AM' '05/26/2023 05:12:00 PM']
Total unique (non-null): 78769
----------------------------------------
Column: Found Location
['9409 Bluegrass Dr in Austin (TX)' '2818 Palomino Trail in Austin (TX)'
 '4404 Saraso

1. Drop Name & Outcome time
2. Convert Age upon intake to single number (in weeks)
3. Split sex upon intake to intact/spayed/neutered and the actual gender (male/female)

In [5]:
test_data = pd.read_csv('test.csv', header=0)
test_data.head()

,Id,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth
0,1,1/3/19 16:19,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,1/3/17
1,2,10/21/13 7:59,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,9/21/13
2,3,6/29/14 10:38,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,6/29/10
3,4,7/11/15 18:19,Galilee Court And Damita Jo Dr in Manor (TX),Stray,Normal,Dog,Intact Female,5 months,Pit Bull,Brown/White,1/11/15
4,5,2/4/17 10:10,208 Beaver St in Austin (TX),Stray,Injured,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black/White,2/4/15


In [10]:
test_data.isnull().sum()

Id                  0
Intake Time         0
Found Location      0
Intake Type         0
Intake Condition    0
Animal Type         0
Sex upon Intake     0
Age upon Intake     0
Breed               0
Color               0
Date of Birth       0
dtype: int64

In [13]:
test_data.shape

(27791, 11)

In [33]:
train_x = train_data.drop('Outcome Type', axis=1)
train_y = train_data['Outcome Type']

0     Return to Owner
1     Return to Owner
2            Transfer
3     Return to Owner
4     Return to Owner
5            Adoption
6     Return to Owner
7            Adoption
8            Transfer
9            Transfer
10           Adoption
11           Adoption
12    Return to Owner
13           Adoption
14           Adoption
15           Adoption
16           Adoption
17           Adoption
18           Adoption
19           Transfer
Name: Outcome Type, dtype: object